In [33]:
# each instance of this class stores a single fittness rule
class Fitness_Rule :  
    # takes a string which containes the rule and builds the rule
    def __init__(self, line) :
        self.option_list = []
        line_list = line.split('#')
        for element in line_list :
            if element.count(':') == 0 :
                self.option_list.append(element)
            else :
                self.option_list.append(element[:element.find(':')])
                self.fitnes_value = float(element[element.find(':')+2:])    
                
    # print overload
    def __str__(self) :
        out = ""
        for element in self.option_list :
            out += element
            out += " ** "
        out = out[:-3]
        out += " : " + str(self.fitnes_value)
        return out
        
    # takes an option dictionariy and returns the cost of this rule
    def get_partial_fitness(self, option_activation) :
        applicable = 1
        for x in self.option_list :
            applicable = applicable and option_activation[x]
        return applicable * self.fitnes_value        

    
# contains all fittness rules
class Fitness_Modell :
    # takes a string containing fittness rules and adds them to the rule list
    def add_fitness_rules(self, fittness_input) :
        for line in fittness_input :
            self.fittness_rule_list.append(Fitness_Rule(line))
            

    # takes a list containing fittness rules and builds the rule list
    def __init__(self) :
        self.fittness_rule_list = []
        
                        
    # print overload
    def __str__(self) :
        out = ""
        for element in self.fittness_rule_list :
            out += element.__str__() + "\n"
        out = out[:-2]
        return out
    
    # takes an option activation dictionary and calculates the fitness(or cost) value
    def calculate_fitness(self, option_activation) : 
        fittness = 0.0
        for element in self.fittness_rule_list :
            fittness += element.get_partial_fitness(option_activation)
        return fittness



# takes a dictionary and re
#def calculate fitness()

In [34]:
# an instance of this class stores all constrains
# TODO: optimize validity check
class Constraint_Model :
        
    def __init__(self) :
        self.constraint_list = []
        self.global_tabu_list = []
        self.mandatory_activation = []
                 
    # input line(string) list
    def build_model(self, dimacs_input) :
        self.constraint_list = []
        self.global_tabu_list = []
        self.mandatory_activation = []
        tmp_constraint = []
        for line in dimacs_input :
            # fill constraint list 
            if len(line) > 0 and line[0] != 'c' and line[0] != 'p' :
                line_list = line.split(' ')
                for element in line_list :
                    if element == '0' and tmp_constraint != [] :
                        self.constraint_list.append(tmp_constraint)
                        tmp_constraint = []
                    elif len(element) > 0 :
                        tmp_constraint.append(int(element))      
            # initialize tabu list and mandatory activation
            elif len(line) > 0 and line[0] == 'p' :
                line_list = line.split(' ')
                self.global_tabu_list = [False] * (int(line_list[2]) + 1)
                self.mandatory_activation = [False] * (int(line_list[2]) + 1)
                
                
    # print function            
    def __str__ (self) :
        out = "Constraints:\n"
        for a in self.constraint_list :
            out += str(a) + "-"+ str(len(a)) + ", "
        
        out += "\n\nGLobal Tabu List:\n"
        out += str(self.global_tabu_list)
        
        out += "\n\nMandatory Activation List:\n"
        out += str(self.mandatory_activation)
           
        return out
        
    
    # builds the global tabu list and sets the activation values 
    def build_global_tabu_list(self) :
        new_constraint_list = []
        for constraint in self.constraint_list :
            self.global_tabu_list[abs(constraint[0])] = True      
            if len(constraint) == 1 :    
                self.global_tabu_list[abs(constraint[0])] = True      
                self.mandatory_activation[abs(constraint[0])] = constraint[0] > 0    
            else :
                new_constraint_list.append(constraint)
                
        self.constraint_list = new_constraint_list
#        for constraint in self.constraint_list :
#            if len(constraint) == 2 :
    
    
    # TODO: optimize !!!!!!
    def check_validity (self, activation_arry) :
        for constraint in self.constraint_list :
            if not self.check_partial_validity(constraint, activation_arry) :
                return False
        return True
    
    # TODO: optimize
    def check_partial_validity (self, constraint, activation_arry) :
        for x in constraint :
            if x > 0 :
                if activation_arry[x] :
                    return true
            else :
                 if activation_arry[x * (-1)] :
                    return True   
        return False    

In [35]:
def linear_regression(n) :
    import itertools
    combinations = list(map(list, itertools.product([0, 1], repeat=n)))
    return(combinations)

def parse_xml_to_dimacs(file):
    var_map = [] #

    final_string = "" #
    variables_string = "" #
    p_line = "p cnf " #
    clauses = "" #
    current_options = [] # 
    optional_false_elements = []
    clauses_2D = []

    from lxml import etree
    xml_rules = etree.parse(open(file, 'r'))
    root = xml_rules.getroot()

    # first iteration to generate the dimacs variables
    for child in root :
        for subchild in child :
            for element in subchild : 
                if element.tag == 'name':
                    var_map.append(element.text)

    # generate variables
    for el in var_map:
        variables_string += "c " + str(1+var_map.index(el)) + " " + el + "\n"

    # generate rules
    for child in root :
        for subchild in child :
            for element in subchild :

                # temporary save current element and excluded options
                if element.tag == 'name':
                    current_options.append(element.text)
                if element.tag == 'excludedOptions' :
                    for option in element :
                        current_options.append(option.text)

                # clear temp if 'otional' tag is set to true
                if element.tag == 'optional' :
                    if element.text != 'False' :
                        current_options = []

            # if there are choices to do generate all combinations at once
            if len(current_options) > 0 :
                ###########################################################################
                # adding all combinations of rules if they are not already in 
                ###########################################################################
                # current_name, current_options
                #
                # print("Inside combinations loop", str(len(current_options)))

                if len(current_options) == 1 :
                    #print("exact one element")
                    for element in current_options:
                        #print("1")
                        tmp=[]
                        tmp.append(1+var_map.index(element))
                        clauses_2D.append(tmp)

                if len(current_options) > 1 :
                    ####################################################current_options = current_options.sort()
                    combinations = linear_regression(len(current_options))
                    for lst in combinations :
                        #print(str(len(current_options)))
                        #print(len(combinations))
                        tmp_list = []
                        for el, el2 in zip(lst, current_options) :
                            if el :
                                tmp_list.append(1+var_map.index(el2))
                            else :
                                tmp_list.append((1+var_map.index(el2))*(-1))

                        # do not put duplicate elements in list
                        tmp_list.sort()
                        is_in_list = False
                        for item in clauses_2D:
                            if item == tmp_list:
                                is_in_list = True
                        if not is_in_list :
                            clauses_2D.append(tmp_list)

            current_options = []

    p_line += str(len(var_map))+" "+str(len(clauses_2D))+"\n"

    #print(clauses_2D)
    for line in clauses_2D :
        for item in line :
            clauses += str(item)+" "
        clauses += "0\n"

    # und zack fertig .... dimacs
    final_string = variables_string+p_line+clauses
    return final_string.split('\n')

In [42]:
class Wrapper :
    
    def __init__(self, path_to_folder, project_name) :
        from os import listdir
        from os.path import isfile, join
        
        
        self.fitness_model = Fitness_Modell()
        self.constraint_model = Constraint_Model()
        
        
        
       
        files = [join(path_to_folder, f) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f))]
        
        for file in files:
             # -------- Fitness Model -----------
            if file.count(project_name) > 0 and (file.count('feature') > 0 or file.count('interactions') > 0) :
                with open(file, 'r') as f:
                    self.fitness_model.add_fitness_rules(f.readlines())
            
             # ------ Constraint Model ---------
            if file.count(project_name) > 0 and file.count('.xml') > 0 :
                self.constraint_model.build_model(parse_xml_to_dimacs(file))
            if file.count(project_name) > 0 and file.count('.dimacs') > 0 :
                with open(file, 'r') as f:
                    self.constraint_model.build_model(f.readlines())           

In [43]:
testinstanz = Wrapper('./project_public_1/', 'h264')
#print(testinstanz.fitness_model)
#print(testinstanz.constraint_model)


['c 1 no_asm', 'c 2 no_8x8dct', 'c 3 no_cabac', 'c 4 no_deblock', 'c 5 no_fast_pskip', 'c 6 no_mbtree', 'c 7 no_mixed_refs', 'c 8 no_weightb', 'c 9 rc_lookahead', 'c 10 rc_lookahead_20', 'c 11 rc_lookahead_40', 'c 12 rc_lookahead_60', 'c 13 ref', 'c 14 ref_1', 'c 15 ref_5', 'c 16 ref_9', 'p cnf 16 19', '1 0', '9 0', '-12 -11 -10 0', '-11 -10 12 0', '-12 -10 11 0', '-10 11 12 0', '-12 -11 10 0', '-11 10 12 0', '-12 10 11 0', '10 11 12 0', '13 0', '-16 -15 -14 0', '-15 -14 16 0', '-16 -14 15 0', '-14 15 16 0', '-16 -15 14 0', '-15 14 16 0', '-16 14 15 0', '14 15 16 0', '']
Constraints:
[1]-1, [9]-1, [-12, -11, -10]-3, [-11, -10, 12]-3, [-12, -10, 11]-3, [-10, 11, 12]-3, [-12, -11, 10]-3, [-11, 10, 12]-3, [-12, 10, 11]-3, [10, 11, 12]-3, [13]-1, [-16, -15, -14]-3, [-15, -14, 16]-3, [-16, -14, 15]-3, [-14, 15, 16]-3, [-16, -15, 14]-3, [-15, 14, 16]-3, [-16, 14, 15]-3, [14, 15, 16]-3, 

GLobal Tabu List:
[False, False, False, False, False, False, False, False, False, False, False, False, Fa